In [12]:
import requests

In [13]:
def get_json_for_state(state):

    url = f'https://static01.nyt.com/elections-assets/2020/data/api/2020-11-03/state-page/{state}.json'
    r = requests.get(url)
    if r.status_code == 200:
        return r.json()
    else:
        print(
            f"Something went wrong with state {state}. I'm outputing the raw response")
        return r

In [14]:
california_json = get_json_for_state('wisconsin')['data']

In [15]:
pres_race = list(filter(lambda x: x["race_name"] == "President", california_json['races']))[0]

In [16]:
pres_race.keys()

dict_keys(['race_id', 'race_slug', 'url', 'state_page_url', 'ap_polls_page', 'edison_exit_polls_page', 'race_type', 'election_type', 'election_date', 'runoff', 'race_name', 'office', 'officeid', 'nyt_race_description', 'race_rating', 'seat', 'seat_name', 'state_id', 'state_slug', 'state_name', 'state_nyt_abbrev', 'state_shape', 'party_id', 'uncontested', 'report', 'result', 'result_source', 'gain', 'lost_seat', 'votes', 'electoral_votes', 'absentee_votes', 'absentee_counties', 'absentee_count_progress', 'absentee_outstanding', 'absentee_max_ballots', 'provisional_outstanding', 'provisional_count_progress', 'poll_display', 'poll_countdown_display', 'poll_waiting_display', 'poll_time', 'poll_time_short', 'precincts_reporting', 'precincts_total', 'reporting_display', 'reporting_value', 'eevp', 'tot_exp_vote', 'eevp_source', 'eevp_value', 'eevp_display', 'county_data_source', 'incumbent_party', 'no_forecast', 'last_updated', 'candidates', 'has_incumbent', 'leader_margin_value', 'leader_mar

In [17]:
pres_timeseries = pres_race["timeseries"]
pres_timeseries[0]

{'vote_shares': {'trumpd': 0, 'bidenj': 0},
 'votes': 0,
 'eevp': 0,
 'eevp_source': 'edison',
 'timestamp': '2020-11-04T10:02:04Z'}

In [18]:
total_djt_diff = 0
total_jrb_diff = 0

In [19]:
def print_discrepancy(before_id):
    global total_djt_diff, total_jrb_diff
    
    before_update = pres_timeseries[before_id]
    after_update = pres_timeseries[before_id+1]
    
    votes_djt_before = round(before_update["vote_shares"]["trumpd"] * before_update["votes"])
    votes_jrb_before = round(before_update["vote_shares"]["bidenj"] * before_update["votes"])
    votes_djt_after = round(after_update["vote_shares"]["trumpd"] * after_update["votes"])
    votes_jrb_after = round(after_update["vote_shares"]["bidenj"] * after_update["votes"])

    diff_djt = votes_djt_after - votes_djt_before
    diff_jrb = votes_jrb_after - votes_jrb_before
    diff_total = after_update["votes"] - before_update["votes"]
    
    total_djt_diff += diff_djt
    total_jrb_diff += diff_jrb
    
    print("DISCREPANCY DETECTED:\n")
    print("DJT VOTES BEFORE: {}".format(votes_djt_before))
    print("JRB VOTES BEFORE: {}".format(votes_jrb_before))
    print("TOTAL BEFORE: {}".format(before_update["votes"]))
    print("TIMESTAMP BEFORE: {}\n".format(before_update["timestamp"]))

    print("DJT VOTES AFTER: {}".format(votes_djt_after))
    print("JRB VOTES AFTER: {}".format(votes_jrb_after))
    print("TOTAL AFTER: {}".format(after_update["votes"]))
    print("TIMESTAMP AFTER: {}\n".format(after_update["timestamp"]))


    print("DJT DIFF: {}".format(diff_djt))
    print("JRB DIFF: {}".format(diff_jrb))
    print("TOTAL DIFF: {}".format(diff_total))
    print("===================================")

In [20]:
for i,update in enumerate(pres_timeseries):
    before_update = pres_timeseries[i]
    after_update = pres_timeseries[i+1]
    
    # Calculate vote counts
    votes_djt_before = round(before_update["vote_shares"]["trumpd"] * before_update["votes"])
    votes_jrb_before = round(before_update["vote_shares"]["bidenj"] * before_update["votes"])
    votes_djt_after = round(after_update["vote_shares"]["trumpd"] * after_update["votes"])
    votes_jrb_after = round(after_update["vote_shares"]["bidenj"] * after_update["votes"])
    
    if((votes_djt_after < votes_djt_before) or (votes_jrb_after < votes_jrb_before)):
        print_discrepancy(i)
    
    if(i==len(pres_timeseries)-2):
        break
    
    

DISCREPANCY DETECTED:

DJT VOTES BEFORE: 172392
JRB VOTES BEFORE: 180293
TOTAL BEFORE: 359150
TIMESTAMP BEFORE: 2020-11-04T02:36:25Z

DJT VOTES AFTER: 172660
JRB VOTES AFTER: 180214
TOTAL AFTER: 359709
TIMESTAMP AFTER: 2020-11-04T02:37:36Z

DJT DIFF: 268
JRB DIFF: -79
TOTAL DIFF: 559
DISCREPANCY DETECTED:

DJT VOTES BEFORE: 312731
JRB VOTES BEFORE: 337194
TOTAL BEFORE: 661164
TIMESTAMP BEFORE: 2020-11-04T02:47:06Z

DJT VOTES AFTER: 312340
JRB VOTES AFTER: 336824
TOTAL AFTER: 661737
TIMESTAMP AFTER: 2020-11-04T02:48:16Z

DJT DIFF: -391
JRB DIFF: -370
TOTAL DIFF: 573
DISCREPANCY DETECTED:

DJT VOTES BEFORE: 399481
JRB VOTES BEFORE: 406863
TOTAL BEFORE: 820289
TIMESTAMP BEFORE: 2020-11-04T02:54:34Z

DJT VOTES AFTER: 400068
JRB VOTES AFTER: 406640
TOTAL AFTER: 821494
TIMESTAMP AFTER: 2020-11-04T02:55:58Z

DJT DIFF: 587
JRB DIFF: -223
TOTAL DIFF: 1205
DISCREPANCY DETECTED:

DJT VOTES BEFORE: 661114
JRB VOTES BEFORE: 630884
TOTAL BEFORE: 1314342
TIMESTAMP BEFORE: 2020-11-04T03:35:25Z

DJT VO

In [21]:
print("TOTAL DJT DIFF: {}".format(total_djt_diff))
print("TOTAL JRB DIFF: {}".format(total_jrb_diff))

TOTAL DJT DIFF: 12382
TOTAL JRB DIFF: -12144
